# Création d'un classeur multi-classes pour jouer aux papier, ciseaux 

Plus souvent qu'autrement, nous sommes intéressés à catégoriser une image en plus de deux classes. Comme nous le verrons dans cette recette, la mise en œuvre d'un réseau de neurones pour différencier de nombreuses catégories est assez simple, et quelle meilleure façon de le démontrer qu'en entraînant un modèle capable de jouer au célèbre jeu Rock Paper Scissors ?

Nous utiliserons le jeu de données Rock-Paper-Scissors Images, qui est hébergé sur Kaggle à l'emplacement suivant : https://www.kaggle.com/drgfreeman/rockpaperscissors. Pour le télécharger, vous aurez besoin d'un compte Kaggle, alors connectez-vous ou inscrivez-vous en conséquence. Ensuite, décompressez l'ensemble de données à l'emplacement de votre choix.

In [1]:
!pip install -q kaggle

Vous devrez télécharger votre fichier kaggle.json pour cette étape, qui peut être obtenu à partir de votre compte Kaggle

In [5]:
from google.colab import files

files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle datasets download -d drgfreeman/rockpaperscissors

 97% 297M/306M [00:03<00:00, 113MB/s]
100% 306M/306M [00:03<00:00, 96.3MB/s]


In [7]:
#!unzip rockpaperscissors.zip

Les étapes suivantes expliquent comment former un réseau de neurones convolutifs (CNN) multi-classes pour faire la distinction entre les trois classes du jeu Rock Paper Scissors

**1.** Importez les packages requis :

In [8]:
import os
import pathlib

import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy

**2.** Dénissez une liste avec les trois classes, ainsi qu'un alias vers tf.data.experimental.AUTOTUNE, que nous utiliserons plus tard :

In [9]:
CLASSES = ['rock', 'paper', 'scissors']
AUTOTUNE = tf.data.experimental.AUTOTUNE

Les valeurs dans CLASSES correspondent aux noms des répertoires qui contiennent les images pour chaque classe

**3.** Dénir une fonction pour charger une image et son label, étant donné son chemin :

In [10]:
def load_image_and_label(image_path, target_size=(32, 32)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)  # One-hot encode.
    label = tf.dtypes.cast(label, tf.float32)

    return image, label

Notez que nous effectuons un encodage one-hot en comparant le nom du dossier qui contient l'image (extrait de image_path) avec la liste CLASSES.

**4.** Déﬁnissez une fonction pour construire l'architecture du réseau. Dans ce cas, c'est très simple et superficiel, ce qui est suffisant pour le problème que nous résolvons :

In [17]:
def build_network():
    input_layer = Input(shape=(32, 32, 1))
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ReLU()(x)
    x = Dropout(rate=0.5)(x)

    x = Flatten()(x)
    x = Dense(units=3)(x)
    output = Softmax()(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

**5.** Définir une fonction, étant donné un chemin vers un ensemble de données, renvoyer une instance tf.data.Dataset d'images et d'étiquettes, par lots et éventuellement mélangés :

In [11]:
def prepare_dataset(dataset_path,
                    buffer_size,
                    batch_size,
                    shuffle=True):
    dataset = (tf.data.Dataset
               .from_tensor_slices(dataset_path)
               .map(load_image_and_label,
                    num_parallel_calls=AUTOTUNE))

    if shuffle:
        dataset.shuffle(buffer_size=buffer_size)

    dataset = (dataset
               .batch(batch_size=batch_size)
               .prefetch(buffer_size=buffer_size))

    return dataset

**6.** Chargez les chemins d'images dans une liste :

In [12]:
file_patten = (pathlib.Path('/content') / 'rps-cv-images' / '*' /
               '*.png')
file_pattern = str(file_patten)
dataset_paths = [*glob.glob(file_pattern)]

**7.** Créez des sous-ensembles d'apprentissage, de test et de validation de chemins d'images :

In [14]:
train_paths, test_paths = train_test_split(dataset_paths,
                                           test_size=0.2,
                                           random_state=999)
train_paths, val_paths = train_test_split(train_paths,
                                          test_size=0.2,
                                          random_state=999)

**8.** Préparez les ensembles de données d'entraînement, de test et de validation :

In [15]:
BATCH_SIZE = 1024
BUFFER_SIZE = 1024

train_dataset = prepare_dataset(train_paths,
                                buffer_size=BUFFER_SIZE,
                                batch_size=BATCH_SIZE)
validation_dataset = prepare_dataset(val_paths,
                                     buffer_size=BUFFER_SIZE,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False)
test_dataset = prepare_dataset(test_paths,
                               buffer_size=BUFFER_SIZE,
                               batch_size=BATCH_SIZE,
                               shuffle=False)

**9.** Instanciez et compilez le modèle :

In [18]:
model = build_network()
model.compile(loss=CategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

**10.** Ajustez le modèle pour 250 époques :

In [19]:
EPOCHS = 250
model.fit(train_dataset,
          validation_data=validation_dataset,
          epochs=EPOCHS)

Epoch 1/250
2/2 [==============================] - 32s 1s/step - loss: 1.1101 - accuracy: 0.3236 - val_loss: 1.1131 - val_accuracy: 0.3143
Epoch 2/250
2/2 [==============================] - 4s 2s/step - loss: 1.0961 - accuracy: 0.3386 - val_loss: 1.0894 - val_accuracy: 0.4171
Epoch 3/250
2/2 [==============================] - 4s 2s/step - loss: 1.0987 - accuracy: 0.3679 - val_loss: 1.0840 - val_accuracy: 0.4800
Epoch 4/250
2/2 [==============================] - 4s 2s/step - loss: 1.0851 - accuracy: 0.4293 - val_loss: 1.0872 - val_accuracy: 0.4286
Epoch 5/250
2/2 [==============================] - 4s 2s/step - loss: 1.0770 - accuracy: 0.4529 - val_loss: 1.0865 - val_accuracy: 0.4314
Epoch 6/250
2/2 [==============================] - 4s 2s/step - loss: 1.0694 - accuracy: 0.4921 - val_loss: 1.0732 - val_accuracy: 0.4857
Epoch 7/250
2/2 [==============================] - 4s 2s/step - loss: 1.0595 - accuracy: 0.5643 - val_loss: 1.0572 - val_accuracy: 0.6257
Epoch 8/250
2/2 [================

**11.** Évaluez le modèle sur l'ensemble de test :

In [20]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Loss: {test_loss}, accuracy: {test_accuracy}')

1/1 [==============================] - 1s 1s/step - loss: 0.6401 - accuracy: 0.9338
Loss: 0.6401192545890808, accuracy: 0.9337899684906006


Après 250 époques, notre réseau atteint une précision d'environ 93,5% sur l'ensemble de test. Comprenons ce que nous venons de faire

Nous avons commencé par définir la liste CLASSES, ce qui nous a permis d'encoder rapidement et en un seul clic les étiquettes de chaque image, en fonction du nom du répertoire où elles étaient contenues, comme nous l'avons observé dans le corps de la fonction load_image_and_label(). Dans cette même fonction, nous lisons une image à partir du disque, la décodons à partir de son format JPEG, la convertissons en niveaux de gris (les informations de couleur ne sont pas nécessaires dans ce problème), puis la redimensionnons à des dimensions plus gérables de 32x32x1.

build_network() crée un CNN très simple et peu profond, comprenant une seule couche convolutive, activée avec ReLU(), suivie d'une sortie, une couche entièrement connectée de trois neurones, correspondant au nombre de catégories dans l'ensemble de données. Parce qu'il s'agit d'une tâche de classification multi-classes, nous utilisons Softmax() pour activer les sorties.

prepare_dataset() exploite la fonction load_image_and_label() dénie précédemment pour convertir les chemins en lots de tenseurs d'image et d'étiquettes encodées à one_hot.

À l'aide des trois fonctions expliquées ici, nous avons préparé trois sous-ensembles de données, dans le but d'entraîner, de valider et de tester le réseau de neurones. Nous avons entraîné le modèle pour 250 époques, en utilisant l'optimiseur Adam et CategoricalCrossentropy(from_logits=True) comme fonction de perte (from_logits=True produit un peu plus de stabilité numérique).

Enfin, nous avons obtenu une précision d'environ 93,5% sur l'ensemble de test. Sur la base de ces résultats, vous pouvez utiliser ce réseau comme composant d'un jeu Rock Paper Scissors pour reconnaître les gestes de la main d'un joueur et réagir en conséquence.